In [1]:
# filename: arse.ipynb
# purpose: parse input dataset files and save to duckdb file

# OHT raw data parsing

## Dataset information
- raw dataset is multiple CSV files in a directory. ex) r"\\BlueServer\공동작업폴더\데이터바우처_OHT\수집csv\newdata"
- CSV filename patterns are "afpLog_YYYY-MM-DD_HHMMSS.csv". ex) r"\\BlueServer\공동작업폴더\데이터바우처_OHT\수집csv\newdata\afpLog_2024-05-29_101339.csv" 
- CSV file does not include head line, comma(,) delimited, 6 columns
- CSV file line example: datatime_ms, tem, ecm1(electronic current measure 1), ecm2, ecm3, ecm4. ex) 2024-05-29 09:13:39:889,34.5,-0.1,1,0.5,0.6
- For reusing performance, a duckdb file will be created with dir-basename.duckdb. ex) r"\\BlueServer\공동작업폴더\데이터바우처_OHT\수집csv\수집csv.duckdb"

## Duckdb information
- raw dataset is multiple CSV files in a directory. ex) r"\\BlueServer\공동작업폴더\데이터바우처_OHT\수집csv\newdata"
- For reusing performance, a duckdb file will be created with parentdir-basename.duckdb. ex) r"\\BlueServer\공동작업폴더\데이터바우처_OHT\수집csv\수집csv.duckdb"
- TABNAME_RAW table was created when the dbfile was created.
- TABNAME_NORM, NOISE, OUTL will be created and used in this script

## Processing flow
- Parse raws csvfiles
- Save to Raw table in-file duckdb
- Display data info

In [2]:
# packages
import time
import pathlib
import pandas as pd

import humanfriendly as human

import ohtconf as conf
import ohtcomm as comm

## Main

In [3]:
mainstart = time.time()

In [4]:
if conf.DBFILE_RECREATE:
    comm.remove_file(conf.DBFILE)

In [5]:
# read dataset files in a directory
_start = time.time()

dfraw: pd.DataFrame = None

if pathlib.Path(conf.DBFILE).exists():
    dfraw = comm.read_tabdf(conf.TABNAME_RAW)
    # ensure order
    dfraw.sort_values(by=conf.COLUMN_NAMES[0], inplace=True)
    dfraw.reset_index(drop=True, inplace=True)

    _elapsed = time.time() - _start
    print(f"Read elapsed time: {human.format_timespan(_elapsed)}")
else:
    csvfiles = comm.get_multifiles_indir(conf.DIRRAW, conf.FILENAME_PATTERN)
    if len(csvfiles) < 1:
        raise Exception(f"dataset no csvfiles in the directory, {conf.DIRRAW}")

    print(f"all csvfile count={len(csvfiles)} in dir={conf.DIRRAW}")

    # skip files
    if conf.SKIP_FILES is not None and conf.SKIP_FILES > 0:
        if len(csvfiles) > conf.SKIP_FILES:
            csvfiles = csvfiles[conf.SKIP_FILES :]
            print(f"skip files={conf.SKIP_FILES} applied")
        else:
            print(f"Ignore SKIP_FILES={conf.SKIP_FILES} as not enough files at dir={conf.DIRRAW}")

    # limit files
    if conf.LIMIT_FILES is not None and conf.LIMIT_FILES > 0:
        if len(csvfiles) > conf.LIMIT_FILES:
            csvfiles = csvfiles[: conf.LIMIT_FILES]
            print(f"limit files={conf.LIMIT_FILES} applied")

    csvsizes = comm.get_multifiles_size(csvfiles)

    if conf.INPUT_MAXSIZE is not None and conf.INPUT_MAXSIZE > 0:
        if sum(csvsizes) > conf.INPUT_MAXSIZE:
            cumfiles, cumsizes, sumsize = [], [], 0
            for fname, fsize in zip(csvfiles, csvsizes):
                cumfiles.append(fname)
                cumsizes.append(fsize)
                sumsize += fsize
                if sumsize > conf.INPUT_MAXSIZE:
                    csvfiles = cumfiles
                    csvsizes = cumsizes
                    break
            print(f"input maxsize={human.format_size(conf.INPUT_MAXSIZE)} applied")

    print(f"selected csvfile count={len(csvfiles)}, size={human.format_size(sum(csvsizes))}")
    print(f"selected first file={csvfiles[0]}, last file={csvfiles[-1]}")

    dfraw = comm.read_multifiles(files=csvfiles, logstep=conf.LOGSTEP_FILES, verbose=conf.VERBOSE)

    # ensure order, non-null
    dfraw.sort_values(by=conf.COLUMN_NAMES[0], inplace=True)
    dfraw.reset_index(drop=True, inplace=True)
    dfraw.bfill(inplace=True)
    dfraw.ffill(inplace=True)

    # save to db
    comm.save_dftab(dfraw, conf.TABNAME_RAW)

    _elapsed = time.time() - _start
    print(f"Parse elapsed time: {human.format_timespan(_elapsed)}")

all csvfile count=279 in dir=C:\projects\ohtdatafiles\dataraw
input maxsize=419.43 MB applied
selected csvfile count=170, size=420.81 MB
selected first file=C:\projects\ohtdatafiles\dataraw\afpLog_2024-07-29_094318.csv, last file=C:\projects\ohtdatafiles\dataraw\afpLog_2024-08-05_091706.csv
file reading 1 file=afpLog_2024-07-29_094318.csv


file reading 11 file=afpLog_2024-07-29_194318.csv


file reading 21 file=afpLog_2024-07-30_054318.csv


file reading 31 file=afpLog_2024-07-30_154318.csv


file reading 41 file=afpLog_2024-07-31_014318.csv


...

file reading 141 file=afpLog_2024-08-04_041706.csv


file reading 151 file=afpLog_2024-08-04_141706.csv


file reading 161 file=afpLog_2024-08-05_001706.csv


dataframe prepared with (rows,columns)=(6095021, 11) in 170 files.


Parse elapsed time: 1 minute and 20.04 seconds


In [6]:
# 데이터 체크
dfraw.head()

,DATETM,TEM,PM1,PM2_5,PM10,CO,NH3,CT1,CT2,CT3,CT4
0,2024-07-29 09:43:18.696,40.799999,10,12,13,161,88,0.8,1.0,0.5,0.6
1,2024-07-29 09:43:18.781,40.799999,10,12,13,161,88,0.8,1.0,0.5,0.6
2,2024-07-29 09:43:18.883,40.799999,10,12,13,161,88,0.8,1.0,0.5,0.5
3,2024-07-29 09:43:18.989,40.799999,10,12,13,161,88,0.9,1.0,0.5,0.6
4,2024-07-29 09:43:19.094,40.799999,10,12,13,161,88,0.8,1.0,0.7,0.6


In [7]:
# 데이터 통계
dfraw.describe()

,DATETM,TEM,PM1,PM2_5,PM10,CO,NH3,CT1,CT2,CT3,CT4
count,6095021,6.095021e+06,6.095021e+06,6.095021e+06,6.095021e+06,6.095021e+06,6.095021e+06,6.095021e+06,6.095021e+06,6.095021e+06,6.095021e+06
mean,2024-08-01 21:56:36.582684928,4.211648e+01,9.383445e+00,1.084684e+01,1.231504e+01,1.572084e+02,9.824781e+01,6.282347e-01,9.729512e-01,4.428262e-01,5.762765e-01
min,2024-07-29 09:43:18.696000,2.260000e+01,7.000000e+00,8.000000e+00,9.000000e+00,3.100000e+01,6.600000e+01,-5.000000e-01,2.000000e-01,2.000000e-01,3.000000e-01
25%,2024-07-31 04:02:54.219000064,4.080000e+01,8.000000e+00,9.000000e+00,1.100000e+01,1.410000e+02,8.800000e+01,3.000000e-01,9.000000e-01,4.000000e-01,5.000000e-01
50%,2024-08-01 21:37:40.710000128,4.170000e+01,9.000000e+00,1.100000e+01,1.200000e+01,1.580000e+02,9.300000e+01,6.000000e-01,1.000000e+00,4.000000e-01,6.000000e-01
75%,2024-08-03 15:57:22.609999872,4.350000e+01,1.000000e+01,1.200000e+01,1.300000e+01,1.700000e+02,1.100000e+02,9.000000e-01,1.000000e+00,5.000000e-01,6.000000e-01
max,2024-08-05 10:17:05.909000,4.550000e+01,2.700000e+01,2.900000e+01,4.200000e+01,2.530000e+02,1.400000e+02,3.000000e+00,1.500000e+00,8.000000e-01,1.000000e+00
std,NaN,1.725541e+00,1.615970e+00,1.644981e+00,1.463494e+00,2.884160e+01,1.239270e+01,3.341508e-01,7.151102e-02,7.401512e-02,6.755878e-02


In [8]:
_elapsed = time.time() - mainstart
print(f"main elapsed time: {human.format_timespan(_elapsed)}")
# 1 min. for 170 files, 420 MB input files when set conf.INPUT_MAXSIZE = 400 MB

main elapsed time: 1 minute and 21.36 seconds


## eof